# Numerical Solution of the Schrödinger–Newton Equation for a Bose–Einstein Condensate (BEC) Halo Using the Thomas Algorithm

The numerical method employed is based on the theoretical framework developed by Peter Jay Salzman in his dissertation *Investigation of the Time Dependent Schrödinger-Newton Equation* (University of California, 2005). [Available here](http://www.dirac.org/physics/dissertation/dissertation.pdf). It has been adapted to a specific potential, as described in the following sections.

In [1]:
import math
import numpy as np

### (Model specific) constants

This section introduces both fundamental and model-specific constants required for the numerical computations. The natural constants include the gravitational constant and the reduced Planck constant. Additionally, parameters defining the dark matter halo - its radius, total mass, and average density - are specified. These values are placeholders and should be updated when applying the method to a physically accurate dark matter model.

In [2]:
# natural constants

G = 6.67430*10**(-11) # Gravitational constant [m^3 kg^-1 s^-2]
h = 1.054571817*10**(-34) # reduced planck constant [J s]

In [3]:
# constants that can (and - for a serious dark matter model - should) be adapted; 
    # as an example, values are proposed that are based on an estimate of the mass and size of the Milky Way

a = 20*3.0857*10**19 # radius of the Dark Matter Halo (FWHM-full width half maximum) [m]
M = 1.37*10**11*1.98847*10**30 # total mass Dark Matter Halo [kg]
rho = M/((4/3)*math.pi*a**3) # average density [kg m^-3]; density = mass/volume

For analytical simplicity, we model the dark matter halo as a homogeneous fluid, which permits the use of a uniform density profile. Under the assumption of a spherically symmetric mass distribution, the resulting gravitational potential takes the form: $$
\Phi(r) = \frac{1}{2} \Omega^2 r^2 + \text{const.}
$$
This form follows the approach presented by Wagner (2020) in Cosmic Structures from a Mathematical Perspective 1: Dark Matter Halo Mass Density Profiles. See: https://doi.org/10.1007/s10714-020-02715-w

In [4]:
om_sq = (4*math.pi*G*rho/3) # harmonic oscillator frequency squared (of this model: Wagner)

### Preparations for numerical calculations

In the following, we define: (1) the spatial and temporal step sizes used in the numerical calculation, (2) three constants introduced to simplify the implementation, and (3) the number of steps taken in both space and time.

The radius r_99, which denotes the region within which 99% of the wave function's total probability density is contained, can only be determined accurately if the spatial resolution $dr$ and/or the number of grid points $N$ are sufficiently large. In this implementation, $dr$ is deliberately set to a large value for computational practicality.

In [5]:
# temporal and spatial steps; can be modified
# dr must be chosen small enough that the probability density of the wave function, integrated over space, always yields approximately 1

dr = 100000000000000000000 # spatial grid size [m] (chosen large enough such that r_99 can be computed)
dt = 2 # temporal grid size [s]

In [6]:
# constants used in the numerical solution of the Schrödinger–Newton equation;
# they appear in the triagonal Matrix

R = dt/dr**2 # constant
K = h/(8*M)*1j # constant
P = dt/(2*h)*1j # constant

In [7]:
# number of temporal and spatial steps; can be modified

N = 300 # number of spatial grid steps
n = 2 # number of temporal grid steps

$N$ and $dr$ must be chosen such that the final calculated  $$ \psi_n^{N-1}$$ is zero. Otherwise the model does not describe the entire DM halo.

### Numerical calculations

1. **Wave function at** $t=0$: In the following, the wave function at time $t = 0$ is loaded into $N\times 1$ vector, $\psi^0_j$, at $N$ distinct spatial grid points. psi\_n\_J is a list in which all **wave function vectors** $\psi^n_j$ for each time step will be saved.\
A spherically symmetric wave function with half-width $a$ is applied at $t=0$, following the paper of Giulini, D., and Großardt, A. Gravitationally induced inhibitions of dispersion according to a modified Schrödinger–Newton equation for a homogeneous-sphere potential. Classical and Quantum Gravity 30, 15 (Jul 2013), 155018. See https://iopscience.iop.org/article/10.1088/0264-9381/30/15/155018/meta.

In [8]:
# 1 preparing psi_n_J and adding the wave vector at t=0 to the list for each spacial step

psi_n_J = [[]] # initial list for wave vectors at timesteps 0...n

for J in range(0, N): # for J = 0, ..., N-1 (not j, since j could be confused with the imaginary number i)
    psi_n_J[0].append((math.pi*a**2)**(-3/4)*math.exp(-(J*dr)**2/(2*a**2)))

print(psi_n_J[0][-1])

0.0


The code below implements steps 2 through 5 of the numerical procedure:

2. **Potential energy at $t=0$**: In the following code, the **potential energy** values $V^n_j$ at each spacial step are calculated for $t=0$. The potential energy, written in pseudo-numerical code, reads $$V_{gj}^n=2\pi\Omega^2\Bigg[(j\Delta r)^2\displaystyle\sum_{i=0}^\infty i^2(\Delta r)^3|\psi_i^n|^2+\displaystyle\sum_{i=0}^\infty i^4 (\Delta r)^5|\psi_i^n|^2\Bigg]\ \ \ \text{.}$$<br><br>

3. **Construction of the tri-diagonal matrix**: The tri-diagonal matrix $$
\begin{bmatrix}
b_0 & c_0 & 0 & 0 & & & & & ... & \\
a_1 & b_1 & c_1 & 0 & & & & & ... & \\
0 & a_2 & b_2 & c_2 & & & & & ... & \\
& & & & . & & & & & \\
& & & & & . & & & & \\
& & & & & & . & & & \\
& & ... & & & & & a_{N-2} & b_{N-2} & c_{N-2} \\
& & ... & & & & & 0 & a_{N-1} & b_{N-1} 
\end{bmatrix}
\begin{bmatrix}
\Upsilon^n_1\\
\Upsilon^n_2\\
\Upsilon^n_3\\
. \\
. \\
. \\
\Upsilon^n_{N-1}\\
\Upsilon^n_N
\end{bmatrix}
=
\begin{bmatrix}
\psi^n_1\\
\psi^n_2\\
\psi^n_3\\
. \\
. \\
. \\
\psi^n_{N-1}\\
\psi^n_N
\end{bmatrix}
$$ which is solved in step 4 using the Thomas algorithm. The coefficients $a_j$, $b_j$ and $c_j$ are loaded into lists A_J, B_J and C_J according to the conditions and formulas outlined in the Bachelor's thesis.<br><br>

4. **Solving the system of equations using the Thomas algorithm**: As detailed in the Bachelor's thesis, additional definitions of factors $B_i$ and $D_i$ are required, which depend on $a_i$, $b_i$, $c_i$, and $\psi_i^n$. The values $\Upsilon^n_i$ are then computed recursively based on these factors. The resulting values are stored in the lists B, D, and X, respectively. <br><br>

5. **Calculating the wave function for each time and spacial step**: Finally, new wave function at time-step $1$ is determined using $\psi^1_j = \Upsilon^0_j - \psi^0_j$ in each case.

In [9]:
# 2. calculating V_0_J from psi_0_J

for t in range(0, n): # going through all temporal steps
    V_n_J = [] # initial empty list for each gravitational energy vector 

    for J in range(0, N): # going through all spatial steps
        sum_1 = 0
        sum_2 = 0
        i = 0
        while i < N: # loop to get the sums within the potential
            sum_1 += i**2*dr**3*abs(psi_n_J[t][i])**2  
            sum_2 += i**4*dr**5*abs(psi_n_J[t][i])**2
            i += 1    
        V_n_J.append(2*math.pi*om_sq*((J*dr)**2)*sum_1-sum_2) # append the for each spatial step the potential energy value
        

# 3. forming the triagonal matrix
    
    A_J = [] # initial list for a_J (used in Thomas Algorithm)
    B_J = [] # initial list for b_J (used in Thomas Algorithm)
    C_J = [] # initial list for c_J (used in Thomas Algorithm)

    for J in range(0, N):
            
        if J == 0:
            b_0 = 0.5*(1+P*V_n_J[t]+12*K*R) # entry
            c_0 = -6*K*R # entry
            A_J.append(0)
            B_J.append(b_0)
            C_J.append(c_0)

        elif J == N-1:
            a_J = -K*R*((N-2)/(N-1)) # entry
            b_J = 0.5*(1+P*V_n_J[t]) # entry
            A_J.append(a_J)
            B_J.append(b_J)
            C_J.append(0)

        else:
            a_J = -K*R*(J-1)/J # entry
            b_J = 0.5*(1+P*V_n_J[t]+2*K*R) # entry
            c_J = -K*R*(J+1)/J # entry
            A_J.append(a_J)
            B_J.append(b_J)
            C_J.append(c_J)

    del(V_n_J) # Vpot for the timestep 0 deleted because it is not needed anymore
    

# 4. solving the triagonal system using the Thomas Algorithm

    a_0 = 0 # convention
    c_Nminus1 = 0 # convention

    X = [] # initial list for the chi^0_J vector (the order of the entries will be reversed later)
    B = [] # initial list for b_i coefficients
    D = [] # initial list for d_i coefficients

    for i in range(0, N):
        if i == 0: # first entry in the help lists
            B.append(B_J[i])
            D.append(psi_n_J[t][i])
        else: #  
            B.append(B_J[i]-A_J[i]*C_J[i-1]/B[i-1])
            D.append(psi_n_J[t][i]-A_J[i]*D[i-1]/B[i-1])

    for i in range(0, N):
        if i == 0:
            X.append(D[N-1-i]/B[N-1-i])
        else:
            X.append((D[N-1-i]-C_J[N-1-i]*X[i-1])/B[N-1-i]) 

    X.reverse()

    del(A_J, B_J, C_J, B, D)


# 5. obtaining new wavefunction 5

    psi_n_J.append(list(np.array(X)-np.array(psi_n_J[t]))) # new wavefunction psi^1_J vector
        
    del(X)


In [10]:
np.array(psi_n_J).shape

(3, 300)

### Calculation of  r_99 

We compute the radius r_99, defined as the radius within which 99% of the total probability density of the wave function is enclosed:

$$
4\pi \, \int_0^{r_{99}} |\psi(r)|^2 \, r^2 dr = 0.99
$$

In the context of a Bose–Einstein condensate dark matter halo described by a single wave function, this radius represents the effective size of the halo — the region containing 99% of the total mass. For comparison, the Milky Way's dark matter halo is estimated to have a radius of about 300,000 light-years, i.e. $2.84×10^{21}$ meters.

In [11]:
r_99 = [] # preparation of the list in which r99 for all timesteps will be saved
l = 0 # preparation of the running index, l = 0 ... n-1 or until s > 0.99

while l <= n: 
    
    i=0
    s=0
    
    for k in psi_n_J[l]:
        
        # calculating the integral that defines r99
        
        s += 4*math.pi*((i)*dr)**2*abs(k)**2*dr # current the sum of all s plus the next
        
        if s > 0.9900000000000: # sum of all summands contributing to s must stay <= 0.99
            r_99.append((i)*dr) # (i)*dr ist the current radius at which this condition is satisfied
            l += 1 # only move to the next l when we've found r_99 for this one
            break
            
        else:
            i += 1 # new radius is old radius + dr; i.e. i*dr = (i-1)*dr + dr
            continue
            
    else:
        # this else belongs to the for-loop and triggers if no break occurred
        # in that case, we still want to increment l to avoid infinite loop
        l += 1

ly0 = r_99[0] / 9.461e15 # calculate the radius r99 in lightyears

print(f"Sum of all summands: {s:.3f}%") 
print(f"r99 at time step 0: {ly0:.2f} light-years")

Sum of all summands: 0.995%
r99 at time step 0: 158545.61 light-years


If we compare this number to the estimated radius of the Milky Way's dark matter halo, we see that it is about half the size for a dark matter halo. This is a realistic estimate.

### Further possible analysis

With greater computational capacity, investigating the time evolution of r_99 could offer deeper insight. At present, r_99 remains unchanged throughout the simulation, likely due to the coarse spatial resolution introduced by the large value of $dr$.

In [12]:
# short impression about r_99

r_99

[1500000000000000000000, 1500000000000000000000, 1500000000000000000000]